In [10]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest


In [12]:
!dvc pull
notebook_dir = os.getcwd()
data_path = os.path.join(notebook_dir, "..", "data", "MachineLearningRating_v3.txt")

df = pd.read_csv(data_path, delimiter="|")

# Add derived columns
df['HasClaim'] = df['TotalClaims'] > 0
df['ClaimFrequency'] = df['HasClaim'].astype(int)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']


Everything is up to date.


C:\Users\Bisrat\AppData\Local\Temp\ipykernel_11456\158655124.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, delimiter="|")


In [13]:
province_claims = df.groupby('Province')['HasClaim'].agg(['sum', 'count'])
province_claims.columns = ['Claims', 'Total']
province_claims['NonClaims'] = province_claims['Total'] - province_claims['Claims']

# Create contingency table
contingency_table = province_claims[['Claims', 'NonClaims']].values

# Chi-squared test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Chi2 = {chi2:.2f}, p = {p:.4f}")

if p < 0.05:
    print("Reject H₀: Risk differs across provinces.")
else:
    print("Fail to reject H₀: No significant difference.")


Chi2 = 104.19, p = 0.0000
Reject H₀: Risk differs across provinces.


In [15]:
# Get top 2 zip codes
top_zips = df['PostalCode'].value_counts().head(2).index.tolist()
zip_data = df[df['PostalCode'].isin(top_zips)]

group_1 = zip_data[zip_data['PostalCode'] == top_zips[0]]['Margin']
group_2 = zip_data[zip_data['PostalCode'] == top_zips[1]]['Margin']

# T-test
t_stat, p_val = stats.ttest_ind(group_1, group_2, equal_var=False)

print(f"T = {t_stat:.2f}, p = {p_val:.4f}")

if p_val < 0.05:
    print(f"Reject H₀: Margin differs between {top_zips[0]} and {top_zips[1]}.")
else:
    print(f"Fail to reject H₀: No margin difference between {top_zips[0]} and {top_zips[1]}.")


T = 1.16, p = 0.2445
Fail to reject H₀: No margin difference between 2000 and 122.


In [16]:
df_claims = df[df['TotalClaims'] > 0]
grouped = df_claims.groupby('VehicleType')['TotalClaims'].apply(list)

# ANOVA
f_stat, p_val = stats.f_oneway(*grouped)

print(f"F = {f_stat:.2f}, p = {p_val:.4f}")

if p_val < 0.05:
    print("Reject H₀: Severity differs across vehicle types.")
else:
    print("Fail to reject H₀: No significant severity difference.")


F = 1.17, p = 0.3228
Fail to reject H₀: No significant severity difference.
